In [1]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pyupbit
import math

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [2]:
class Data_preprocess() :
    def __init__(self, ticker, interval, to, count, norm) :
        self.norm = {"minmax" : self.MinMax,
                    "stand" : self.standarization,
                    "diff" : self.diff}
        
        self.data, self.label, self.dataset = self.preprocess(pyupbit.get_ohlcv(ticker=ticker, interval=interval, to=to, count=count),
                                                             normalization=norm)
        
    def MinMax(self, df) :
        norm = MinMaxScaler()
        norm_dataset = norm.fit_transform(df)
        return pd.DataFrame(norm_dataset, columns=list(df.columns))
    
    def standarization(self, df) :
        for col in df:
            df[col] = (df[col] - df[col].mean()) / df[col].std()
        return df
    
    def diff(self, df) :
        for col in (df):
            log_y = np.log(df[col])
            df[col][1:] = np.diff(log_y)
        return df[1:]
    

    def add_label(self, dataset_df) :
        after10 = np.zeros_like(dataset_df['close'])
        for i in range(len(dataset_df['close']) - 1) :
            if dataset_df['close'][i + 1] > dataset_df['close'][i] :
                after10[i] = 1
            else : 
                after10[i] = 0
            
        return pd.DataFrame(after10,columns=['label'])
    
    
    def drop_feature(self, dataset_df) :
        # index(시간) 제거
        dataset_df = dataset_df.reset_index(drop=True)
        # value 제거
#         dataset_df = dataset_df.drop(columns=['value'])
        return dataset_df
    
    
    def add_avgPrice(self, dataset_df) :
        return (dataset_df['high'] + dataset_df['low'] + 
                dataset_df['open'] + dataset_df['close']) // 4
       
    
    def preprocess(self, dataset, normalization) :
        
        # drop feature
        dataset_df = self.drop_feature(dataset)
        
        # avg_price 추가
        dataset_df['avg_price'] = self.add_avgPrice(dataset_df)
        
        # label 추가
        if normalization == "diff" :
            label = self.add_label(dataset_df)[1:-1]
        else :
            label = self.add_label(dataset_df)[:-1]
        
        norm_df = self.norm[normalization](dataset_df.copy())[:-1]
        
        return norm_df, label, dataset_df[:-1]
        
        

In [3]:
ticker = 'KRW-BTC'
interval ='minute10'
to = f'2022-03-29 00:00'
count = 288000

processed_data =  Data_preprocess(ticker, interval, to, count, "stand")
display(processed_data.data)
display(processed_data.label)
display(processed_data.dataset)

,open,high,low,close,volume,value,avg_price
0,-0.854681,-0.853338,-0.854432,-0.853480,-0.472213,-0.496971,-0.853985
1,-0.854029,-0.853199,-0.853731,-0.853480,-0.469498,-0.496541,-0.853613
2,-0.854029,-0.853292,-0.854618,-0.853759,-0.470259,-0.496697,-0.853927
3,-0.853843,-0.853292,-0.854105,-0.852781,-0.453105,-0.494194,-0.853508
4,-0.853470,-0.853710,-0.854245,-0.853620,-0.441865,-0.492584,-0.853764
...,...,...,...,...,...,...,...
236104,1.613274,1.607807,1.605662,1.602654,0.695077,2.058569,1.607356
236105,1.602747,1.601996,1.607856,1.602328,-0.077394,0.548755,1.603735
236106,1.602281,1.600648,1.600948,1.606194,0.091435,0.876071,1.602524
236107,1.606147,1.602461,1.606969,1.601769,-0.255885,0.200173,1.604340


,label
0,0.0
1,0.0
2,1.0
3,0.0
4,0.0
...,...
236104,0.0
236105,1.0
236106,0.0
236107,1.0


,open,high,low,close,volume,value,avg_price
0,4201000.0,4241000.0,4195000.0,4227000.0,15.940028,6.722878e+07,4216000.0
1,4215000.0,4244000.0,4210000.0,4227000.0,16.224769,6.854317e+07,4224000.0
2,4215000.0,4242000.0,4191000.0,4221000.0,16.144995,6.806705e+07,4217250.0
3,4219000.0,4242000.0,4202000.0,4242000.0,17.943481,7.572972e+07,4226250.0
4,4227000.0,4233000.0,4199000.0,4224000.0,19.122032,8.065820e+07,4220750.0
...,...,...,...,...,...,...,...
236104,57184000.0,57185000.0,56901000.0,56955000.0,138.327126,7.890153e+09,57056250.0
236105,56958000.0,57060000.0,56948000.0,56948000.0,57.335712,3.268367e+09,56978500.0
236106,56948000.0,57031000.0,56800000.0,57031000.0,75.036941,4.270333e+09,56952500.0
236107,57031000.0,57070000.0,56929000.0,56936000.0,38.621467,2.201300e+09,56991500.0


In [4]:
processed_data.dataset.to_csv('../data/288000_data.csv')
processed_data.label.to_csv('../data/288000_label.csv')


In [2]:
csv_data = pd.read_csv('../data/144000_data.csv').drop(columns=['Unnamed: 0'])
csv_label = pd.read_csv('../data/144000_label.csv').drop(columns=['Unnamed: 0'])

In [6]:
def EachColumnMinMax(df) :
    norm = MinMaxScaler()
    df = norm.fit_transform(np.array(df).reshape(-1, 1)).squeeze(1)
    return df

def EachColumnStand(df) :
    df = (df - df.mean()) / df.std()
    return df

def EachColumnDiff(df) :
    log_y = np.log(df)
    df[1:] = np.diff(log_y)
    return df[1:]


def WindowDataGenerator(df_data, df_label, window_size, stride, norm) :
    
    if norm == "diff" :
        num_sample = ((df_data.shape[0] - 1) - window_size) // stride + 1
        data = np.zeros([window_size - 1, df_data.shape[1], num_sample])
    else : 
        num_sample = (df_data.shape[0] - window_size) // stride + 1
        data = np.zeros([window_size, df_data.shape[1], num_sample])

    
    labels = np.zeros([num_sample])

    for i in tqdm(range(num_sample)) :
        tmp = {}
        data_start = stride * i
        data_end = data_start + window_size
        
        for col in df_data.columns :
            if norm == "diff" :
                tmp[col] = EachColumnDiff(df_data[col][data_start : data_end])
            elif norm == "stand" :
                tmp[col] = EachColumnStand(df_data[col][data_start : data_end])
            elif norm == "minmax" :
                tmp[col] = EachColumnMinMax(df_data[col][data_start : data_end])

        data[:, :, i] = pd.DataFrame(tmp).values
        labels[i] = df_label.values[data_end - 1]
        
        
    data = data.transpose((2, 0, 1))
    print("dataset shape ==== ",data.shape)
    
    # data shape (80, 600, 6), label shape (80,)
    return torch.Tensor(data), torch.Tensor(labels)

In [7]:
# display(csv_data[1:-1].shape)

# a,b = WindowDataGenerator(csv_data[:-1], csv_label, 24 * 6, 117, "diff")
# print(a.shape)
# print(b.shape)

In [3]:
class Transformer2FC(nn.Module) :
    def __init__(self, input_shape, d_model, n_head, num_layer, dropout, num_class=2):
        super(Transformer2FC, self).__init__()
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layer)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        self.Encoder = nn.Sequential(
            nn.Linear(input_shape[1], d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, d_model)
        )
        
        self.linear = nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.ReLU(),
            nn.Linear(d_model//2, 1)
        )

        self.linear2 = nn.Sequential(
            nn.Linear(input_shape[0], input_shape[0]//2),
            nn.ReLU(),
            nn.Linear(input_shape[0]//2, num_class)
        )
        
#         self.sigmoid = nn.Softmax()
    
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    
    def forward(self, x, masked_x) :
        # (batch, data, dim)
#         print("input shape : ", x.shape)
        x = self.Encoder(x)
#         print("Encoder shape : ", x.shape)
        x = self.pos_encoder(x)
#         print("pos_encoder shape : ", x.shape)
#         print("masked_x shape : ", masked_x.shape)
        x = self.transformer_encoder(x.transpose(0,1), masked_x).transpose(0, 1)
#         print("transformer_encoder shape : ", x.shape)
        x = self.linear(x)
#         print("linear shape : ", x.shape)
        x = x.squeeze(2)
#         print("squeeze shape : ", x.shape)
        x = self.linear2(x)
#         print("linear2 shape : ", x.shape)
        x = x.squeeze(1)
#         print("squeeze shape : ", x.shape)
        return x

class PositionalEncoding(nn.Module) :
    def __init__(self, d_model, dropout=0.1, max_len=5000) :
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)
        
    def forward(self, x) :
        x = x + self.pe[:x.size(0), :]

        return self.dropout(x)

def get_attention_mask(x) :
    mask = torch.eq(x, 0)
    return mask

In [4]:
class WindowDataset(Dataset) :
    def __init__(self, df_data, df_label, window_size=144, stride=6, norm="diff") :
        self.data, self.label = self.WindowDataGenerator(df_data, df_label, window_size, stride, norm)
        
    def __getitem__(self, i) :
        return self.data[i], self.label[i]
                
    def __len__(self) :
        assert len(self.data) == len(self.label), "data와 label의 길이가 다름"
        return len(self.data)
        
    
    def EachColumnMinMax(self, df) :
        norm = MinMaxScaler()
        df = norm.fit_transform(np.array(df).reshape(-1, 1)).squeeze(1)
        return df

    def EachColumnStand(self, df) :
        df = (df - df.mean()) / df.std()
        return df

    def EachColumnDiff(self, df) :
        log_y = np.log(df)
        df[1:] = np.diff(log_y)
        return df[1:]


    def WindowDataGenerator(self, df_data, df_label, window_size, stride, norm) :
        if norm == "diff" :
            num_sample = ((df_data.shape[0] - 1) - window_size) // stride + 1
            data = np.zeros([window_size - 1, df_data.shape[1], num_sample])
        else : 
            num_sample = (df_data.shape[0] - window_size) // stride + 1
            data = np.zeros([window_size, df_data.shape[1], num_sample])
            
        labels = np.zeros([num_sample])

        for i in tqdm(range(num_sample)) :
            tmp = {}
            data_start = stride * i
            data_end = data_start + window_size
            for col in df_data.columns :
                if norm == "diff" :
                    tmp[col] = self.EachColumnDiff(df_data[col][data_start : data_end].copy())
                elif norm == "stand" :
                    tmp[col] = self.EachColumnStand(df_data[col][data_start : data_end].copy())
                elif norm == "minmax" :
                    tmp[col] = self.EachColumnMinMax(df_data[col][data_start : data_end].copy())

#                 tmp[col] = self.EachColumnMinMax(df_data.loc[data_start : data_end - 1, col])

            data[:, :, i] = pd.DataFrame(tmp).values
            labels[i] = df_label.values[data_end - 1]

        data = data.transpose((2, 0, 1))
        print("dataset shape ==== ",data.shape)

        # data shape (80, 600, 6), label shape (80,)
        return torch.Tensor(data), torch.Tensor(labels)

In [5]:
device = torch.device("cuda:0")
lr = 1e-4
epochs = 500
window_size = 24 * 6
window_stride = 12
feature_len = 7
batch_size = 128
num_class = 1

model = Transformer2FC(input_shape=(window_size, feature_len), 
                       d_model=512, 
                       n_head=8, 
                       num_layer=4, 
                       dropout=0.3, 
                       num_class = num_class).to(device)
# model = MLSTMfcn(max_seq_len=window_size, num_features=feature_len).to(device)
criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss()
# criterion = nn.HuberLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [6]:
dataset = WindowDataset(csv_data, 
                        csv_label,
                       window_size = window_size, 
                       stride = window_stride, norm="stand")

dataloader = DataLoader(dataset, batch_size=batch_size)

100%|███████████████████████████████████████████████████████████████████████████| 11988/11988 [00:52<00:00, 228.53it/s]

dataset shape ====  (11988, 144, 7)


In [7]:
def cal_accuracy(pred, label, threshold=0) :
    acc = torch.zeros(pred.shape[0])
    acc[pred > threshold] = 1
    acc[pred < threshold] = 0
    score = [1 if acc[i] == label[i] else 0 for i in range(pred.shape[0])]
    return sum(score) / pred.shape[0]

In [8]:
# progress = tqdm(range(epoch))
mean_loss = []
mean_acc = []

model.train()
for e in range(epochs) :
    batchloss = 0
    batchacc= 0
    tqdm_dataloader = tqdm(dataloader)
    for batch, (data, label) in enumerate(tqdm_dataloader, start=1) :
        optim.zero_grad()
        src_mask = model.generate_square_subsequent_mask(data.shape[1]).to(device)      
        
        pred = model(data.to(device), src_mask)
        
        loss = criterion(pred, label.to(device, dtype=torch.float64))

        loss.backward()
        optim.step()
        
        score = cal_accuracy(pred.cpu(), label.cpu())
        
        batchloss += loss.cpu().item()
        batchacc += score        
        
        tqdm_dataloader.set_description(f"Epoch[{e}/{epochs}]   mean loss : {batchloss/batch:0.7f}   mean acc : {batchacc/batch:0.7f}")

    mean_loss.append(batchloss/batch)
    mean_acc.append(batchacc/batch)

    
torch.save(model.state_dict(), f'./{epochs}E_stand_model.pt')

Epoch[0/500]   mean loss : 0.6936983   mean acc : 0.5092373: 100%|█████████████████████| 94/94 [00:31<00:00,  3.01it/s]
Epoch[1/500]   mean loss : 0.6933019   mean acc : 0.5047928: 100%|█████████████████████| 94/94 [00:31<00:00,  2.97it/s]
Epoch[2/500]   mean loss : 0.6932199   mean acc : 0.5068468: 100%|█████████████████████| 94/94 [00:32<00:00,  2.92it/s]
Epoch[3/500]   mean loss : 0.6930321   mean acc : 0.5067004: 100%|█████████████████████| 94/94 [00:32<00:00,  2.90it/s]
Epoch[4/500]   mean loss : 0.6932335   mean acc : 0.5102069: 100%|█████████████████████| 94/94 [00:33<00:00,  2.82it/s]
Epoch[5/500]   mean loss : 0.6930441   mean acc : 0.5057466: 100%|█████████████████████| 94/94 [00:34<00:00,  2.71it/s]
Epoch[6/500]   mean loss : 0.6931260   mean acc : 0.5086159: 100%|█████████████████████| 94/94 [00:33<00:00,  2.81it/s]
Epoch[7/500]   mean loss : 0.6931648   mean acc : 0.5065935: 100%|█████████████████████| 94/94 [00:33<00:00,  2.78it/s]
Epoch[8/500]   mean loss : 0.6928288   m

Epoch[68/500]   mean loss : 0.6574719   mean acc : 0.5980639: 100%|████████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[69/500]   mean loss : 0.6567771   mean acc : 0.5967737: 100%|████████████████████| 94/94 [00:37<00:00,  2.51it/s]
Epoch[70/500]   mean loss : 0.6538646   mean acc : 0.6025955: 100%|████████████████████| 94/94 [00:36<00:00,  2.59it/s]
Epoch[71/500]   mean loss : 0.6519352   mean acc : 0.6010955: 100%|████████████████████| 94/94 [00:36<00:00,  2.60it/s]
Epoch[72/500]   mean loss : 0.6518768   mean acc : 0.6023817: 100%|████████████████████| 94/94 [00:36<00:00,  2.60it/s]
Epoch[73/500]   mean loss : 0.6486421   mean acc : 0.6023065: 100%|████████████████████| 94/94 [00:35<00:00,  2.67it/s]
Epoch[74/500]   mean loss : 0.6466603   mean acc : 0.6057537: 100%|████████████████████| 94/94 [00:35<00:00,  2.64it/s]
Epoch[75/500]   mean loss : 0.6453840   mean acc : 0.6129330: 100%|████████████████████| 94/94 [00:35<00:00,  2.65it/s]
Epoch[76/500]   mean loss : 0.6460321   

Epoch[136/500]   mean loss : 0.5438196   mean acc : 0.7027530: 100%|███████████████████| 94/94 [00:36<00:00,  2.61it/s]
Epoch[137/500]   mean loss : 0.5411794   mean acc : 0.7024957: 100%|███████████████████| 94/94 [00:36<00:00,  2.61it/s]
Epoch[138/500]   mean loss : 0.5430478   mean acc : 0.6986409: 100%|███████████████████| 94/94 [00:35<00:00,  2.62it/s]
Epoch[139/500]   mean loss : 0.5431919   mean acc : 0.7017992: 100%|███████████████████| 94/94 [00:35<00:00,  2.63it/s]
Epoch[140/500]   mean loss : 0.5395853   mean acc : 0.7041580: 100%|███████████████████| 94/94 [00:35<00:00,  2.63it/s]
Epoch[141/500]   mean loss : 0.5374361   mean acc : 0.7037899: 100%|███████████████████| 94/94 [00:35<00:00,  2.63it/s]
Epoch[142/500]   mean loss : 0.5375249   mean acc : 0.7019139: 100%|███████████████████| 94/94 [00:36<00:00,  2.61it/s]
Epoch[143/500]   mean loss : 0.5317895   mean acc : 0.7103557: 100%|███████████████████| 94/94 [00:36<00:00,  2.60it/s]
Epoch[144/500]   mean loss : 0.5316780  

Epoch[204/500]   mean loss : 0.4462168   mean acc : 0.7702159: 100%|███████████████████| 94/94 [00:37<00:00,  2.52it/s]
Epoch[205/500]   mean loss : 0.4404943   mean acc : 0.7757013: 100%|███████████████████| 94/94 [00:37<00:00,  2.53it/s]
Epoch[206/500]   mean loss : 0.4386292   mean acc : 0.7744546: 100%|███████████████████| 94/94 [00:36<00:00,  2.54it/s]
Epoch[207/500]   mean loss : 0.4341259   mean acc : 0.7797302: 100%|███████████████████| 94/94 [00:36<00:00,  2.55it/s]
Epoch[208/500]   mean loss : 0.4312088   mean acc : 0.7771063: 100%|███████████████████| 94/94 [00:37<00:00,  2.54it/s]
Epoch[209/500]   mean loss : 0.4298224   mean acc : 0.7816379: 100%|███████████████████| 94/94 [00:37<00:00,  2.50it/s]
Epoch[210/500]   mean loss : 0.4349422   mean acc : 0.7762039: 100%|███████████████████| 94/94 [00:37<00:00,  2.51it/s]
Epoch[211/500]   mean loss : 0.4325761   mean acc : 0.7789070: 100%|███████████████████| 94/94 [00:36<00:00,  2.58it/s]
Epoch[212/500]   mean loss : 0.4277103  

Epoch[272/500]   mean loss : 0.3659332   mean acc : 0.8195526: 100%|███████████████████| 94/94 [00:36<00:00,  2.55it/s]
Epoch[273/500]   mean loss : 0.3643073   mean acc : 0.8223270: 100%|███████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[274/500]   mean loss : 0.3668053   mean acc : 0.8218679: 100%|███████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[275/500]   mean loss : 0.3663477   mean acc : 0.8177162: 100%|███████████████████| 94/94 [00:37<00:00,  2.50it/s]
Epoch[276/500]   mean loss : 0.3637775   mean acc : 0.8213375: 100%|███████████████████| 94/94 [00:36<00:00,  2.58it/s]
Epoch[277/500]   mean loss : 0.3631153   mean acc : 0.8216265: 100%|███████████████████| 94/94 [00:36<00:00,  2.57it/s]
Epoch[278/500]   mean loss : 0.3624223   mean acc : 0.8222082: 100%|███████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[279/500]   mean loss : 0.3606522   mean acc : 0.8191331: 100%|███████████████████| 94/94 [00:37<00:00,  2.52it/s]
Epoch[280/500]   mean loss : 0.3633686  

Epoch[340/500]   mean loss : 0.2963625   mean acc : 0.8580769: 100%|███████████████████| 94/94 [00:36<00:00,  2.57it/s]
Epoch[341/500]   mean loss : 0.2976076   mean acc : 0.8611441: 100%|███████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[342/500]   mean loss : 0.3006484   mean acc : 0.8609383: 100%|███████████████████| 94/94 [00:36<00:00,  2.55it/s]
Epoch[343/500]   mean loss : 0.3001036   mean acc : 0.8600241: 100%|███████████████████| 94/94 [00:36<00:00,  2.61it/s]
Epoch[344/500]   mean loss : 0.2917108   mean acc : 0.8635543: 100%|███████████████████| 94/94 [00:37<00:00,  2.54it/s]
Epoch[345/500]   mean loss : 0.2847843   mean acc : 0.8667165: 100%|███████████████████| 94/94 [00:37<00:00,  2.54it/s]
Epoch[346/500]   mean loss : 0.2982540   mean acc : 0.8577365: 100%|███████████████████| 94/94 [00:36<00:00,  2.59it/s]
Epoch[347/500]   mean loss : 0.2927684   mean acc : 0.8643538: 100%|███████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[348/500]   mean loss : 0.2860763  

Epoch[408/500]   mean loss : 0.2374415   mean acc : 0.8944758: 100%|███████████████████| 94/94 [00:36<00:00,  2.58it/s]
Epoch[409/500]   mean loss : 0.2395293   mean acc : 0.8932331: 100%|███████████████████| 94/94 [00:36<00:00,  2.59it/s]
Epoch[410/500]   mean loss : 0.2290406   mean acc : 0.8972937: 100%|███████████████████| 94/94 [00:38<00:00,  2.46it/s]
Epoch[411/500]   mean loss : 0.2366467   mean acc : 0.8958927: 100%|███████████████████| 94/94 [00:37<00:00,  2.54it/s]
Epoch[412/500]   mean loss : 0.2423599   mean acc : 0.8897345: 100%|███████████████████| 94/94 [00:36<00:00,  2.60it/s]
Epoch[413/500]   mean loss : 0.2222047   mean acc : 0.9005430: 100%|███████████████████| 94/94 [00:35<00:00,  2.65it/s]
Epoch[414/500]   mean loss : 0.2329622   mean acc : 0.8951803: 100%|███████████████████| 94/94 [00:36<00:00,  2.58it/s]
Epoch[415/500]   mean loss : 0.2244153   mean acc : 0.8993399: 100%|███████████████████| 94/94 [00:36<00:00,  2.55it/s]
Epoch[416/500]   mean loss : 0.2224168  

Epoch[476/500]   mean loss : 0.1811277   mean acc : 0.9219106: 100%|███████████████████| 94/94 [00:35<00:00,  2.61it/s]
Epoch[477/500]   mean loss : 0.1789958   mean acc : 0.9232404: 100%|███████████████████| 94/94 [00:36<00:00,  2.55it/s]
Epoch[478/500]   mean loss : 0.1802761   mean acc : 0.9234066: 100%|███████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[479/500]   mean loss : 0.1673740   mean acc : 0.9288485: 100%|███████████████████| 94/94 [00:37<00:00,  2.50it/s]
Epoch[480/500]   mean loss : 0.1770084   mean acc : 0.9235293: 100%|███████████████████| 94/94 [00:36<00:00,  2.56it/s]
Epoch[481/500]   mean loss : 0.1907343   mean acc : 0.9177550: 100%|███████████████████| 94/94 [00:37<00:00,  2.49it/s]
Epoch[482/500]   mean loss : 0.1772827   mean acc : 0.9234422: 100%|███████████████████| 94/94 [00:37<00:00,  2.53it/s]
Epoch[483/500]   mean loss : 0.1825944   mean acc : 0.9220333: 100%|███████████████████| 94/94 [00:35<00:00,  2.63it/s]
Epoch[484/500]   mean loss : 0.1945773  

KeyboardInterrupt: 

In [ ]:
mean_loss

In [ ]:
mean_acc

# 학습 성공 config

In [ ]:
ticker = 'KRW-BTC'
interval ='minute10'
to = f'2022-03-29 00:00'
count = 14400 # 144000 가 한계임 288000은 학습이 안됨

lr = 1e-4
epochs = 300
window_size = 24 * 6
window_stride = 12
feature_len = 7
batch_size = 128
num_class = 1

norm = "stand"